In [ ]:
# This script imports the genomic coordinates of region of interest (in this case boundaries of segmental amplifications) 
# and calculate if they are enriched in a list of genomic features

# NOTE
# The code is made available for transparency reasons. At present, it is not intended to be readily usable on different datasets. 
# Also, it was not annotated and compiled to be user-friendly. Please, contact me privately for any inquiry related to the code usage.
# I will maintain this code with improved versions as soon as they are developed.

In [24]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import cm
import glob, os
import sys
import seaborn as sns
from scipy.stats import mode
from matplotlib.patches import Rectangle
import math
import pandas as pd
import itertools
from scipy.fftpack import rfft, irfft, fftfreq
import sys
from numpy import NaN, Inf, arange, isscalar, asarray, array
from __future__ import division
#from scipy.signal import find_peaks

%matplotlib inline
import scipy; print(scipy.__version__)

0.17.0


In [25]:
# Import the lists with the coordinates of various genomic features
repeats = pd.read_csv("repetitive_elements.csv")
fragile = pd.read_csv("fragile_sites.csv")
features = pd.read_csv("features.csv")
trans = pd.read_csv("transcription.csv")
G4 = pd.read_csv("G4.csv")
H2A = pd.read_csv("gH2AX.csv")
RRM = pd.read_csv("rrm3.csv")
rep = pd.read_csv("repetitive.csv")
term = pd.read_csv("terminations_wt.csv")
term2 = pd.read_csv("terminations_fachinetti.csv")
MAT = pd.read_csv("MAT.csv")

In [26]:
# Import the list of CNVs boundaries as brkzones
brkzones = pd.read_excel("breakpoints.xlsx")
chi = pd.DataFrame(columns= ['feature', 'observed in', 'observed out', 'expected in', 'expected out'])

In [27]:
# find the portion of fork stalling areas
genome = 12000000
percentage_break = (len(brkzones)*3000) *100 / (genome *100)

In [28]:
#calculate how many fragile sites are in the proximity of a breakzone
hits = 0
for i in range (16):
    fragile_chr = fragile.loc[fragile['chr'] == (i+1)]
    fragile_chr = fragile_chr.reset_index()
    brkzones_chr = brkzones.loc[brkzones['chr'] == (i+1)]
    brkzones_chr = brkzones_chr.reset_index()
    for n in range(len(fragile_chr)):
        for p in range(len(brkzones_chr)):
            if (fragile_chr['left'][n] < brkzones_chr.start[p] < fragile_chr['right'][n]) or (fragile_chr['left'][n] < brkzones_chr.end[p] < fragile_chr['right'][n])or (brkzones_chr.start[p] < fragile_chr['left'][n] < brkzones_chr.end[p])or (brkzones_chr.start[p] < fragile_chr['right'][n] < brkzones_chr.end[p]):
                hits = hits +1

total_fragile = len(fragile['left'])-5 #-5 to remove the ones in telomeres
expected_fragile = total_fragile * percentage_break
chi.loc[0] = ['fragile sites', hits, (total_fragile - hits), expected_fragile, (total_fragile-expected_fragile)]

In [29]:
#calculate how many Ty elements are included in breakpoint regions
ty = repeats.loc[repeats['type'] == 'LTR retrotransposon']
hits_ty = 0
for i in range (16):
    ty_chr = ty.loc[ty['chr'] == (i+1)]
    ty_chr = ty_chr.reset_index()
    brkzones_chr = brkzones.loc[brkzones['chr'] == (i+1)]
    brkzones_chr = brkzones_chr.reset_index()
    for n in range(len(ty_chr)):
        for p in range(len(brkzones_chr)):
            if (ty_chr['start'][n] < brkzones_chr.start[p] < ty_chr['end'][n]) or (ty_chr['start'][n] < brkzones_chr.end[p] < ty_chr['end'][n])or (brkzones_chr.start[p] < ty_chr['start'][n] < brkzones_chr.end[p])or (brkzones_chr.start[p] < ty_chr['end'][n] < brkzones_chr.end[p]):
                hits_ty = hits +1
                
total_ty = len(ty['start'])-1 #-1 to remove the ones in telomeres
expected_ty = total_ty * percentage_break
chi.loc[1] = ['ty', hits_ty, (total_ty - hits_ty), expected_ty, (total_ty-expected_ty)]

In [30]:
#calculate how many LTRs are included in breakpoint regions
LTR = repeats.loc[repeats['type'] == 'long terminal repeat']
hits_LTR = 0
for i in range (16):
    LTR_chr = LTR.loc[LTR['chr'] == (i+1)]
    LTR_chr = LTR_chr.reset_index()
    brkzones_chr = brkzones.loc[brkzones['chr'] == (i+1)]
    brkzones_chr = brkzones_chr.reset_index()
    for n in range(len(LTR_chr)):
        for p in range(len(brkzones_chr)):
            if (LTR_chr['start'][n] < brkzones_chr.start[p] < LTR_chr['end'][n]) or (LTR_chr['start'][n] < brkzones_chr.end[p] < LTR_chr['end'][n]) or (brkzones_chr.start[p] < LTR_chr['start'][n] < brkzones_chr.end[p])or (brkzones_chr.start[p] < LTR_chr['end'][n] < brkzones_chr.end[p]):
                hits_LTR = hits +1
                
total_LTR = len(LTR['start'])-2 #-2 to remove the ones in telomeres
expected_LTR = total_LTR * percentage_break
chi.loc[2] = ['LTRs', hits_LTR, (total_LTR - hits_LTR), expected_LTR, (total_LTR-expected_LTR)]

In [31]:
#calculate how many tRNA are included in breakpoint regions
tRNA = repeats.loc[repeats['type'] == 'tRNA gene']
hits_tRNA = 0
for i in range (16):
    tRNA_chr = tRNA.loc[tRNA['chr'] == (i+1)]
    tRNA_chr = tRNA_chr.reset_index()
    brkzones_chr = brkzones.loc[brkzones['chr'] == (i+1)]
    brkzones_chr = brkzones_chr.reset_index()
    for n in range(len(tRNA_chr)):
        for p in range(len(brkzones_chr)):
            if (tRNA_chr['start'][n] < brkzones_chr.start[p] < tRNA_chr['end'][n]) or (tRNA_chr['start'][n] < brkzones_chr.end[p] < tRNA_chr['end'][n]) or (brkzones_chr.start[p] < tRNA_chr['start'][n] < brkzones_chr.end[p]) or (brkzones_chr.start[p] < tRNA_chr['end'][n] < brkzones_chr.end[p]):
                hits_tRNA = hits_tRNA +1
                
total_tRNA = len(tRNA['start'])-2 #-2 to remove the ones in telomeres
expected_tRNA = total_tRNA * percentage_break
chi.loc[3] = ['tRNA', hits_tRNA, (total_tRNA - hits_tRNA), expected_tRNA, (total_tRNA-expected_tRNA)]

In [32]:
#calculate how many ARS are included in breakpoint regions
ARS = features.loc[features['type'] == 'ARS'
hits_ARS = 0
for i in range (16):
    ARS_chr = ARS.loc[ARS['chr'] == (i+1)]
    ARS_chr = ARS_chr.reset_index()
    brkzones_chr = brkzones.loc[brkzones['chr'] == (i+1)]
    brkzones_chr = brkzones_chr.reset_index()
    for n in range(len(ARS_chr)):
        for p in range(len(brkzones_chr)):
            if (ARS_chr['start'][n] < brkzones_chr.start[p] < ARS_chr['end'][n])or (ARS_chr['start'][n] < brkzones_chr.end[p] < ARS_chr['end'][n])or (brkzones_chr.start[p] < ARS_chr['start'][n] < brkzones_chr.end[p])or (brkzones_chr.start[p] < ARS_chr['end'][n] < brkzones_chr.end[p]):
                hits_ARS = hits_ARS +1

total_ARS = len(ARS['start'])
expected_ARS = total_ARS * percentage_break
chi.loc[4] = ['ARS', hits_ARS, (total_ARS - hits_ARS), expected_ARS, (total_ARS-expected_ARS)]

In [33]:
#calculate how many snoRNA are included in breakpoint regions
snoRNA = features.loc[features['type'] == 'snoRNA gene']
hits_snoRNA = 0
for i in range (16):
    snoRNA_chr = snoRNA.loc[snoRNA['chr'] == (i+1)]
    snoRNA_chr = snoRNA_chr.reset_index()
    brkzones_chr = brkzones.loc[brkzones['chr'] == (i+1)]
    brkzones_chr = brkzones_chr.reset_index()
    for n in range(len(snoRNA_chr)):
        if len(snoRNA_chr) == 1:
            for p in range(len(brkzones_chr)):
                if (snoRNA_chr['start'][n] < brkzones_chr.start[p] < snoRNA_chr['end'][n]) or (snoRNA_chr['start'][n] < brkzones_chr.end[p] < snoRNA_chr['end'][n]) or (brkzones_chr.start[p] < snoRNA_chr['start'][n] < brkzones_chr.end[p]) or (brkzones_chr.start[p] < snoRNA_chr['end'][n] < brkzones_chr.end[p]):
                    hits_snoRNA = hits_snoRNA +1      
        elif len(snoRNA_chr) ==0:
            print 'zero'
        else:
            for p in range(len(brkzones_chr)):
                if (snoRNA_chr['start'][n] < brkzones_chr.start[p] < snoRNA_chr['end'][n]) or (snoRNA_chr['start'][n] < brkzones_chr.end[p] < snoRNA_chr['end'][n]) or (brkzones_chr.start[p] < snoRNA_chr['start'][n] < brkzones_chr.end[p]) or (brkzones_chr.start[p] < snoRNA_chr['end'][n] < brkzones_chr.end[p]):
                    hits_snoRNA = hits_snoRNA +1

total_snoRNA = len(snoRNA['start'])
expected_snoRNA = total_snoRNA * percentage_break
chi.loc[5] = ['snoRNA', hits_snoRNA, (total_snoRNA - hits_snoRNA), expected_snoRNA, (total_snoRNA-expected_snoRNA)]

In [34]:
#calculate how many nsRNA are included in breakpoint regions

snRNA = features.loc[features['type'] == 'snRNA gene']
hits_snRNA = 0
for i in range (16):
    snRNA_chr = snRNA.loc[snRNA['chr'] == (i+1)]
    snRNA_chr = snRNA_chr.reset_index()
    brkzones_chr = brkzones.loc[brkzones['chr'] == (i+1)]
    brkzones_chr = brkzones_chr.reset_index()
    for n in range(len(snRNA_chr)):
        if len(snRNA_chr) == 1:
            for p in range(len(brkzones_chr)):
                if (snRNA_chr['start'][n] < brkzones_chr.start[p] < snRNA_chr['end'][n])or (snRNA_chr['start'][n] < brkzones_chr.end[p] < snRNA_chr['end'][n])  or (brkzones_chr.start[p] < snRNA_chr['start'][n] < brkzones_chr.end[p])  or (brkzones_chr.start[p] < snRNA_chr['end'][n] < brkzones_chr.end[p]):
                    hits_snRNA = hits_snRNA +1      
        elif len(snRNA_chr) ==0:
            print 'zero'
        else:
            for p in range(len(brkzones_chr)):
                if (snRNA_chr['start'][n] < brkzones_chr.start[p] < snRNA_chr['end'][n])or (snRNA_chr['start'][n] < brkzones_chr.end[p] < snRNA_chr['end'][n])  or (brkzones_chr.start[p] < snRNA_chr['start'][n] < brkzones_chr.end[p])  or (brkzones_chr.start[p] < snRNA_chr['end'][n] < brkzones_chr.end[p]):
                    hits_snRNA = hits_snRNA +1

total_snRNA = len(snRNA['start'])
expected_snRNA = total_snRNA * percentage_break
chi.loc[6] = ['snRNA', hits_snRNA, (total_snRNA - hits_snRNA), expected_snRNA, (total_snRNA-expected_snRNA)]

In [35]:
#calculate how many centromeres are included in breakpoint regions
CEN = features.loc[features['type'] == 'centromere']
hits_CEN = 0
for i in range (16):
    CEN_chr = CEN.loc[CEN['chr'] == (i+1)]
    CEN_chr = CEN_chr.reset_index()
    brkzones_chr = brkzones.loc[brkzones['chr'] == (i+1)]
    brkzones_chr = brkzones_chr.reset_index()
    for n in range(len(CEN_chr)):
        if len(CEN_chr) == 1:
            for p in range(len(brkzones_chr)):
                if (CEN_chr['start'][n] < brkzones_chr.start[p] < CEN_chr['end'][n]) or (CEN_chr['start'][n] < brkzones_chr.end[p] < CEN_chr['end'][n]) or (brkzones_chr.start[p] < CEN_chr['start'][n] < brkzones_chr.end[p])or (brkzones_chr.start[p] < CEN_chr['end'][n] < brkzones_chr.end[p]):
                    hits_CEN = hits_CEN +1      
        elif len(CEN_chr.center) ==0:
            print 'zero'
        else:
            for p in range(len(brkzones_chr)):
                if (CEN_chr['start'][n] < brkzones_chr.start[p] < CEN_chr['end'][n]) or (CEN_chr['start'][n] < brkzones_chr.end[p] < CEN_chr['end'][n]) or (brkzones_chr.start[p] < CEN_chr['start'][n] < brkzones_chr.end[p])or (brkzones_chr.start[p] < CEN_chr['end'][n] < brkzones_chr.end[p]):
                    hits_CEN = hits_CEN +1

total_CEN = len(CEN['start'])
expected_CEN = total_CEN * percentage_break
chi.loc[7] = ['Centromeres', hits_CEN, (total_CEN - hits_CEN), expected_CEN, (total_CEN-expected_CEN)]

In [36]:
#calculate how many highly transcribed genes  are included in breakpoint regions
high = trans[:325]
hits_high = 0
for i in range (16):
    high_chr = high.loc[high['chr'] == (i+1)]
    high_chr = high_chr.reset_index()
    brkzones_chr = brkzones.loc[brkzones['chr'] == (i+1)]
    brkzones_chr = brkzones_chr.reset_index()
    for n in range(len(high_chr)):
        for p in range(len(brkzones_chr)):
            if (high_chr['start'][n] < brkzones_chr.start[p] < high_chr['end'][n]) or (high_chr['start'][n] < brkzones_chr.end[p] < high_chr['end'][n]) or (brkzones_chr.start[p] < high_chr['start'][n] < brkzones_chr.end[p]) or (brkzones_chr.start[p] < high_chr['end'][n] < brkzones_chr.end[p]):
                hits_high = hits_high +1
                
total_high = len(high['start'])
expected_high = total_high * percentage_break
chi.loc[8] = ['Highly transcribed genes', hits_high, (total_high - hits_high), expected_high, (total_high-expected_high)]

In [37]:
#calculate how many weakly trascribed genes are included in breakpoint regions
low = trans[6251:]
hits_low = 0
for i in range (16):
    low_chr = low.loc[low['chr'] == (i+1)]
    low_chr = low_chr.reset_index()
    brkzones_chr = brkzones.loc[brkzones['chr'] == (i+1)]
    brkzones_chr = brkzones_chr.reset_index()
    for n in range(len(low_chr)):
        for p in range(len(brkzones_chr)):
            if (low_chr['start'][n] < brkzones_chr.start[p] < low_chr['end'][n]) or (low_chr['start'][n] < brkzones_chr.end[p] < low_chr['end'][n]) or (brkzones_chr.start[p] < low_chr['start'][n] < brkzones_chr.end[p])or (brkzones_chr.start[p] < low_chr['end'][n] < brkzones_chr.end[p]):
                hits_low = hits_low +1

total_low = len(low['start'])
expected_low = total_low * percentage_break
chi.loc[9] = ['Weakly transcribed genes', hits_low, (total_low - hits_low), expected_low, (total_low-expected_low)]

In [38]:
#calculate how many G4 sequences are included in breakpoint regions
hits_G4 = 0
for i in range (16):
    G4_chr = G4.loc[G4['chr'] == (i+1)]
    G4_chr = G4_chr.reset_index()
    brkzones_chr = brkzones.loc[brkzones['chr'] == (i+1)]
    brkzones_chr = brkzones_chr.reset_index()
    for n in range(len(G4_chr)):
        for p in range(len(brkzones_chr)):
            if (G4_chr['start'][n] < brkzones_chr.start[p] < G4_chr['end'][n]) or (G4_chr['start'][n] < brkzones_chr.end[p] < G4_chr['end'][n]) or (brkzones_chr.start[p] < G4_chr['start'][n] < brkzones_chr.end[p])or (brkzones_chr.start[p] < G4_chr['end'][n] < brkzones_chr.end[p]):
                hits_G4 = hits_G4 +1
                
total_G4 = len(G4['start'])
expected_G4 = total_G4 * percentage_break
chi.loc[10] = ['G4 sequences', hits_G4, (total_G4 - hits_G4), expected_G4, (total_G4-expected_G4)]

In [39]:
#calculate how many gamma H2AX sequences are included in breakpoint regions
hits_H2A = 0
for i in range (16):
    H2A_chr = H2A.loc[H2A['chr'] == (i+1)]
    H2A_chr = H2A_chr.reset_index()
    brkzones_chr = brkzones.loc[brkzones['chr'] == (i+1)]
    brkzones_chr = brkzones_chr.reset_index()
    for n in range(len(H2A_chr)):
        for p in range(len(brkzones_chr)):
            if (H2A_chr['start'][n] < brkzones_chr.start[p] < H2A_chr['end'][n]) or (H2A_chr['start'][n] < brkzones_chr.end[p] < H2A_chr['end'][n]) or (brkzones_chr.start[p] < H2A_chr['start'][n] < brkzones_chr.end[p]) or (brkzones_chr.start[p] < H2A_chr['end'][n] < brkzones_chr.end[p]):
                hits_H2A = hits_H2A +1

total_H2A = len(H2A['start'])
expected_H2A = total_H2A * percentage_break
chi.loc[11] = ['gammaH2AX', hits_H2A, (total_H2A - hits_H2A), expected_H2A, (total_H2A-expected_H2A)]

In [40]:
#calculate how many RRM3 are included in breakpoint regions
hits_RRM = 0
RRM['start'] = RRM['peak']-2500
RRM['end'] = RRM['peak']+2500
for i in range (16):
    RRM_chr = RRM.loc[RRM['chr'] == (i+1)]
    RRM_chr = RRM_chr.reset_index()
    brkzones_chr = brkzones.loc[brkzones['chr'] == (i+1)]
    brkzones_chr = brkzones_chr.reset_index()
    for n in range(len(RRM_chr)):
        if len(RRM_chr) == 1:
            for p in range(len(brkzones_chr)):
                if (RRM_chr['start'][n] < brkzones_chr.start[p] < RRM_chr['end'][n]) or (RRM_chr['start'][n] < brkzones_chr.end[p] < RRM_chr['end'][n]) or (brkzones_chr.start[p] < RRM_chr['start'][n] < brkzones_chr.end[p]) or (brkzones_chr.start[p] < RRM_chr['end'][n] < brkzones_chr.end[p]):
                    hits_RRM = hits_RRM +1      
        elif len(RRM_chr) ==0:
            print 'zero'
        else:
            for p in range(len(brkzones_chr)):
                if (RRM_chr['start'][n] < brkzones_chr.start[p] < RRM_chr['end'][n]) or (RRM_chr['start'][n] < brkzones_chr.end[p] < RRM_chr['end'][n]) or (brkzones_chr.start[p] < RRM_chr['start'][n] < brkzones_chr.end[p]) or (brkzones_chr.start[p] < RRM_chr['end'][n] < brkzones_chr.end[p]):
                    hits_RRM = hits_RRM +1

total_RRM = len(RRM['peak'])
expected_RRM = total_RRM * percentage_break
chi.loc[12] = ['RRM3 binding', hits_RRM, (total_RRM - hits_RRM), expected_RRM, (total_RRM-expected_RRM)]

In [41]:
#calculate how many repetitive sequences are included in breakpoint regions
rep25 = rep.loc[rep['pattern'] >24]
hits_rep25 = 0
for i in range (16):
    rep25_chr = rep25.loc[rep25['chr'] == (i+1)]
    rep25_chr = rep25_chr.reset_index()
    brkzones_chr = brkzones.loc[brkzones['chr'] == (i+1)]
    brkzones_chr = brkzones_chr.reset_index()
    for n in range(len(rep25_chr)):
        if len(rep25_chr) == 1:
            for p in range(len(brkzones_chr)):
                if (rep25_chr['start'][n] < brkzones_chr.start[p] < rep25_chr['end'][n]) or (rep25_chr['start'][n] < brkzones_chr.end[p] < rep25_chr['end'][n]) or (brkzones_chr.start[p] < rep25_chr['start'][n] < brkzones_chr.end[p]) or (brkzones_chr.start[p] < rep25_chr['end'][n] < brkzones_chr.end[p]):
                    hits_rep25 = hits_rep25 +1      
        elif len(rep25_chr) ==0:
            print 'zero'
        else:
            for p in range(len(brkzones_chr)):
                if (rep25_chr['start'][n] < brkzones_chr.start[p] < rep25_chr['end'][n]) or (rep25_chr['start'][n] < brkzones_chr.end[p] < rep25_chr['end'][n]) or (brkzones_chr.start[p] < rep25_chr['start'][n] < brkzones_chr.end[p]) or (brkzones_chr.start[p] < rep25_chr['end'][n] < brkzones_chr.end[p]):
                    hits_rep25 = hits_rep25 +1

total_rep25 = len(rep25['start'])
expected_rep25 = total_rep25 * percentage_break
chi.loc[13] = ['repetitive sequences', hits_rep25, (total_rep25 - hits_rep25), expected_rep25, (total_rep25-expected_rep25)]

In [42]:
#calculate how many terminations sequences are included in breakpoint regions
hits_term = 0
term['start'] = term['position']-2500
term['end'] = term['position']+2500

for i in range (16):
    term_chr = term.loc[term['chr'] == (i+1)]
    term_chr = term_chr.reset_index()
    brkzones_chr = brkzones.loc[brkzones['chr'] == (i+1)]
    brkzones_chr = brkzones_chr.reset_index()
    for n in range(len(term_chr)):
        for p in range(len(brkzones_chr)):
            if (term_chr['start'][n] < brkzones_chr.start[p] < term_chr['end'][n]) or (term_chr['start'][n] < brkzones_chr.end[p] < term_chr['end'][n]) or (brkzones_chr.start[p] < term_chr['start'][n] < brkzones_chr.end[p])or (brkzones_chr.start[p] < term_chr['end'][n] < brkzones_chr.end[p]):
                    hits_term = hits_term +1

total_term = (len(term['position']))
expected_term = (total_term * percentage_break)
chi.loc[14] = ['Replication terminations', hits_term, (total_term - hits_term), expected_term, (total_term-expected_term)]

In [43]:
#calculate how many gamma terminations sequences are included in breakpoint regions
hits_term2 = 0
for i in range (16):
    term2_chr = term2.loc[term2['chr'] == (i+1)]
    term2_chr = term2_chr.reset_index()
    brkzones_chr = brkzones.loc[brkzones['chr'] == (i+1)]
    brkzones_chr = brkzones_chr.reset_index()
    for n in range(len(term2_chr.position)):
        for p in range(len(brkzones_chr)):
            if brkzones_chr.start[p] < term2_chr['position'][n] < brkzones_chr.end[p]:
                hits_term2 = hits_term2 +1

total_term2 = (len(term2['position']))
expected_term2 = (total_term2 * percentage_break)
chi.loc[15] = ['Replication terminations ChIP', hits_term2, (total_term2 - hits_term2), expected_term2, (total_term2-expected_term2)]

In [44]:
#calculate how many MAT sequences are included in breakpoint regions
hits_MAT = 0
for i in range (16):
    MAT_chr = MAT.loc[MAT['chr'] == (i+1)]
    MAT_chr = MAT_chr.reset_index()
    brkzones_chr = brkzones.loc[brkzones['chr'] == (i+1)]
    brkzones_chr = brkzones_chr.reset_index()
    for n in range(len(MAT_chr)):
        if len(MAT_chr) == 1:
            for p in range(len(brkzones_chr)):
                if (MAT_chr['start'][n] < brkzones_chr.start[p] < MAT_chr['end'][n]) or (MAT_chr['start'][n] < brkzones_chr.end[p] < MAT_chr['end'][n]) or (brkzones_chr.start[p] < MAT_chr['start'][n] < brkzones_chr.end[p]) or (brkzones_chr.start[p] < MAT_chr['end'][n] < brkzones_chr.end[p]):
                    hits_MAT = hits_MAT +1      
        elif len(MAT_chr) ==0:
            print 'zero'
        else:
            for p in range(len(brkzones_chr)):
                if (MAT_chr['start'][n] < brkzones_chr.start[p] < MAT_chr['end'][n]) or (MAT_chr['start'][n] < brkzones_chr.end[p] < MAT_chr['end'][n]) or (brkzones_chr.start[p] < MAT_chr['start'][n] < brkzones_chr.end[p]) or (brkzones_chr.start[p] < MAT_chr['end'][n] < brkzones_chr.end[p]):
                    hits_MAT = hits_MAT +1  

total_MAT = len(MAT['start'])
expected_MAT = total_MAT * percentage_break
chi.loc[16] = ['MAT loci', hits_MAT, (total_MAT - hits_MAT), expected_MAT, (total_MAT-expected_MAT)]

In [46]:
# Caluclate chi2 and p values
from scipy.stats import chisquare
from scipy.stats import chisqprob
chi['chi2']= chisquare([chi['observed in'], chi['observed out']], f_exp=[chi['expected in'],chi['expected out']])[0]
chi['pVal']= chisquare([chi['observed in'], chi['observed out']], f_exp=[chi['expected in'],chi['expected out']])[1]

In [47]:
# Function to caluclate false discovery rates
def fdr(p_vals):
    from scipy.stats import rankdata
    ranked_p_values = rankdata(p_vals)
    fdr = p_vals * len(p_vals) / ranked_p_values
    fdr[fdr > 1] = 1
    return fdr

In [48]:
# Calculate the false discovery rates
chi['pVal - False discovery rate'] = fdr(chi['pVal'])

,feature,observed in,observed out,expected in,expected out,chi2,pVal,pVal - False discovery rate
0,fragile sites,16.0,180.0,1.81300,194.18700,112.051907,3.480768e-26,1.479326e-25
1,ty,17.0,32.0,0.45325,48.54675,609.710279,1.293493e-134,2.198938e-133
2,LTRs,17.0,364.0,3.52425,377.47575,52.008593,5.525829e-13,1.878782e-12
3,tRNA,20.0,253.0,2.52525,270.47475,122.054412,2.245931e-28,1.272694e-27
4,ARS,4.0,348.0,3.25600,348.74400,0.171592,6.787007e-01,7.691941e-01
5,snoRNA,0.0,77.0,0.71225,76.28775,0.718900,3.965050e-01,5.185066e-01
6,snRNA,0.0,6.0,0.05550,5.94450,0.056018,8.129038e-01,8.129038e-01
7,Centromeres,0.0,16.0,0.14800,15.85200,0.149382,6.991269e-01,7.428223e-01
8,Highly transcribed genes,4.0,321.0,3.00625,321.99375,0.331562,5.647406e-01,6.857565e-01
9,Weakly transcribed genes,0.0,325.0,3.00625,321.99375,3.034317,8.152083e-02,1.979792e-01


In [49]:
# Correct for multiple testing using Benjamini-Hochberg
positive_bh=chi.sort_values(by='pVal')
positive_bh=positive_bh.reset_index()
positive_bh2=positive_bh.loc[positive_bh['pVal'] < (0.05*(positive_bh.index+1)/len(chi.feature)), :]

,index,feature,observed in,observed out,expected in,expected out,chi2,pVal,pVal - False discovery rate
0,1,ty,17.0,32.0,0.45325,48.54675,609.710279,1.293493e-134,2.198938e-133
1,16,MAT loci,2.0,1.0,0.02775,2.97225,141.480590,1.263149e-32,1.073677e-31
2,3,tRNA,20.0,253.0,2.52525,270.47475,122.054412,2.245931e-28,1.272694e-27
3,0,fragile sites,16.0,180.0,1.81300,194.18700,112.051907,3.480768e-26,1.479326e-25
4,2,LTRs,17.0,364.0,3.52425,377.47575,52.008593,5.525829e-13,1.878782e-12


In [50]:
chi.to_excel("CNVs_genomic_features.xlsx")